In [1]:
library(survey)
library(tidyverse)

Loading required package: grid
Loading required package: Matrix
Loading required package: survival

Attaching package: ‘survey’

The following object is masked from ‘package:graphics’:

    dotchart

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1.9000     ✔ purrr   0.2.4     
✔ tibble  1.3.4          ✔ dplyr   0.7.4     
✔ tidyr   0.7.2          ✔ stringr 1.2.0     
✔ readr   1.1.1          ✔ forcats 0.2.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_chloride_abnormal,lab_ph_flag,lab_ph_first,lab_ph_min,lab_ph_max,lab_ph_abnormal,sedative,echo_int,mort_28_day_int,ps
201220,125078,66690,0,2106-04-27 09:47:50,2106-05-01 19:25:46,1,62.67646,1,NA,⋯,1,1,7.45,7.42,7.45,1,1,0,1,0.4447784
215842,151232,11663,0,2188-02-14 09:48:15,2188-02-16 03:02:48,1,86.76186,1,NA,⋯,0,0,NA,NA,NA,NA,0,0,0,0.1580691
234312,164444,86645,1,2165-06-22 09:47:16,2165-07-07 21:55:20,1,56.08904,1,NA,⋯,1,1,7.49,7.49,7.49,1,0,0,0,0.3088600
289157,146726,10304,0,2156-06-23 22:26:00,2156-06-30 17:26:00,1,45.91093,1,2156-06-25 08:00:00,⋯,0,1,7.44,7.44,7.44,1,0,1,0,0.7000591
211964,160170,94534,0,2160-03-05 22:23:19,2160-03-07 06:48:41,1,59.38693,1,NA,⋯,0,0,NA,NA,NA,NA,0,0,0,0.3324814
230173,112553,31544,0,2140-01-31 04:39:25,2140-02-03 02:41:39,1,300.00345,1,NA,⋯,1,0,NA,NA,NA,NA,0,0,1,0.2379659


In [4]:
table_one <- data.table::fread(file.path(data_dir, "tableone_raw.csv"), data.table = FALSE)
head(table_one)

p_val,wtd_p_val,main_0,main_1,sub_0,sub_1,wtd_main_0,wtd_main_1,wtd_sub_0,wtd_sub_1,var
1.942281e-03,0.04772662,84.183688,79.166557,67.137454,59.7201585,83.089515,79.918203,64.906984,60.763406,age
1.416420e-02,0.34205147,50.822846,47.665687,NA,NA,49.415944,48.205909,NA,NA,female
6.091011e-02,0.03565708,1.801389,1.837418,0.738580,0.7696856,2.104114,2.159730,1.036885,1.040762,weight
1.705518e-15,0.08790062,8.505970,10.045054,7.451124,7.6816088,9.106113,9.436513,7.555664,7.638541,elix_score
1.378591e-15,0.12425987,19.632462,20.778975,5.788347,5.4462928,20.019478,20.236554,5.657332,5.425510,saps
7.217556e-27,0.02783258,5.304937,6.323865,3.618100,3.7963648,5.662260,5.868751,3.669506,3.697442,sofa


In [5]:
wpv <- table_one %>% filter(wtd_p_val < 0.05) %>% pull(var)
wpv

[1] "age"                        "weight"                    
 [3] "sofa"                       "vent"                      
 [5] "vaso"                       "sedative"                  
 [7] "icd_chf"                    "icd_afib"                  
 [9] "lab_chloride_first"         "lab_bnp_flag"              
[11] "lab_troponin_flag"          "lab_creatinine_kinase_flag"

In [6]:
design_echo_ps_ate <- svydesign(ids = ~ icustay_id, weights = ~ weight, data = full_data)

In [7]:
fml <- wpv %>%
c("echo", .) %>%
paste(collapse = " + ") %>%
sprintf("mort_28_day ~ %s", .)

fml

[1] "mort_28_day ~ echo + age + weight + sofa + vent + vaso + sedative + icd_chf + icd_afib + lab_chloride_first + lab_bnp_flag + lab_troponin_flag + lab_creatinine_kinase_flag"

In [8]:
logi <- svyglm(as.formula(fml), family = quasibinomial, design = design_echo_ps_ate)

In [9]:
summary(logi)


Call:
svyglm(formula = as.formula(fml), family = quasibinomial, design = design_echo_ps_ate)

Survey design:
svydesign(ids = ~icustay_id, weights = ~weight, data = full_data)

Coefficients:
                              Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 -1.9155978  0.5166831  -3.707 0.000211 ***
echo1                       -0.2121892  0.0710812  -2.985 0.002846 ** 
age                          0.0033462  0.0005005   6.685 2.52e-11 ***
weight                       0.1539966  0.0430853   3.574 0.000354 ***
sofa                         0.2047693  0.0110825  18.477  < 2e-16 ***
vent1                        0.4635076  0.1093295   4.240 2.27e-05 ***
vaso1                        0.0256861  0.0832122   0.309 0.757574    
sedative1                   -0.4635574  0.1085199  -4.272 1.97e-05 ***
icd_chf1                    -0.0416273  0.0830629  -0.501 0.616281    
icd_afib1                    0.2759192  0.0806980   3.419 0.000632 ***
lab_chloride_first          

In [10]:
exp(cbind(OR = coef(logi), confint(logi)))

,OR,2.5 %,97.5 %
(Intercept),0.1472538,0.05348907,0.4053852
echo1,0.8088116,0.70362776,0.9297192
age,1.0033518,1.00236794,1.0043366
weight,1.1664869,1.07202670,1.2692703
sofa,1.2272419,1.20087204,1.2541909
vent1,1.5896400,1.28303150,1.9695193
vaso1,1.0260188,0.87161549,1.2077741
sedative1,0.6290419,0.50851908,0.7781295
icd_chf1,0.9592273,0.81511385,1.1288201
icd_afib1,1.3177414,1.12496735,1.5435491
